In [ ]:
!pip install simpletransformers

     |████████████████████████████████| 248 kB 5.2 MB/s 
     |████████████████████████████████| 311 kB 33.3 MB/s 
     |████████████████████████████████| 3.5 MB 42.6 MB/s 
     |████████████████████████████████| 9.9 MB 28.4 MB/s 
     |████████████████████████████████| 1.2 MB 35.4 MB/s 
     |████████████████████████████████| 1.7 MB 19.0 MB/s 
     |████████████████████████████████| 6.5 MB 32.2 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
     |████████████████████████████████| 596 kB 41.7 MB/s 
     |████████████████████████████████| 67 kB 5.3 MB/s 
     |████████████████████████████████| 895 kB 42.6 MB/s 
     |████████████████████████████████| 144 kB 45.5 MB/s 
     |████████████████████████████████| 181 kB 45.6 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 212 kB 51.0 MB/s 
     |████████████████████████████████| 1.1 MB 41.4 MB/s 
     |████████████████████████████████| 134 kB 50.9 MB/s 
     |███████████████

In [ ]:
import torch 
from transformers import AutoModelForSequenceClassification,AutoTokenizer,Trainer
from transformers import Trainer
import pandas as pd
import numpy as np
import math


In [ ]:
#torch Dataset

class TestDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
        # self.labels = labels
        # self.size = size

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

In [ ]:
data = pd.read_csv("/content/conversation_with_viewpoint.csv")
# data.head()
data.pop("tweet_id")
data.pop("Unnamed: 0")
data=data.rename(columns={"Unnamed: 0.1":"tweet_id"})
data.head()

,tweet_id,text,relevance,viewpoint
0,1255911223251484672,"@sandynigeluca @JoeSchm19964270 @CNN Yes, it i...",no,NaN
1,1254835941241323520,@MetallicaDad @arjay_bob @jhb430 @thehill I be...,yes,diagnostic
2,1255179026123165697,@Dr_Ezekwesili LMFAO! We thank God for growth ...,no,NaN
3,1255277756990066692,@ProgressiveAnd2 @204cRuiz @FlyThaiMMA @niktay...,no,NaN
4,1255300688101412866,"@dahlingitsme True, I was tweeting from inside...",no,NaN


In [ ]:
test_texts = data["text"].tolist()
tweet_ids = data.tweet_id.to_list()

In [ ]:
#skip tweets that don't have text
for n,i in enumerate(test_texts):
  if (type(i)==float):
    test_texts[n]=""

In [ ]:
#model uploaded on gdrive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#load the model and tokenizer

from transformers import AutoConfig
bert_model = "vinai/bertweet-base"
# bert_model = "berttweet"
# config = AutoConfig.from_pretrained("/content/drive/MyDrive/ml_incivility_intolerance.zip (Unzipped Files)/ml_incivility_intolerance/checkpoint-4254-epoch-3/tokenizer_config.json") 
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/ml_incivility_intolerance.zip (Unzipped Files)/ml_incivility_intolerance/checkpoint-4254-epoch-3")
tokenizer = AutoTokenizer.from_pretrained("roberta-large", config="/content/drive/MyDrive/ml_incivility_intolerance.zip (Unzipped Files)/ml_incivility_intolerance/checkpoint-4254-epoch-3/tokenizer_config.json")

Some weights of the model checkpoint at /content/drive/MyDrive/ml_incivility_intolerance.zip (Unzipped Files)/ml_incivility_intolerance/checkpoint-4254-epoch-3 were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
text_encodings = tokenizer(test_texts, truncation=True, padding=True) #tokenize the tweets

In [ ]:
type(text_encodings)

transformers.tokenization_utils_base.BatchEncoding

In [ ]:
trainer = Trainer(model=model) #initialize the trainer

In [ ]:
# x = TestDataset(text_encodings)
# for i in x:
#   # print(i)
#   if(len(i['input_ids']) != 128):
#     print(len(i['input_ids']))
#   # break

In [ ]:
preds = trainer.predict(TestDataset(text_encodings)) #get predictions from the model

***** Running Prediction *****
  Num examples = 25131
  Batch size = 8


In [ ]:
# np.save("preds.npy",preds)
preds = np.load("preds.npy",allow_pickle=True)

In [ ]:
#sigmoid function
def sigmoid(x):
  z = 1/(1 + np.exp(-x))
  return z

In [ ]:
#pass predictions through sigmoid -- multiclass classification

labels=[]
for i in preds[0]:
  # print(i)
  # break
  y=sigmoid(i)
  z=np.round(y)
  labels.append(z)

In [ ]:
tlist = list(zip(*labels))

In [ ]:
len(tlist[0])

25131

In [ ]:
data["incivility_intolerance"]=labels

In [ ]:
#assign the list to the Label column in the dataframe.

data["profanity"] = tlist[0]
data["insults"] = tlist[1]
data["char. assn."] = tlist[2]
data["outrage"] = tlist[3]
data["L2_intolerance"] = tlist[4]
data["discrimination"] = tlist[5]
data["intolerance"] = tlist[6]
data["incivility"] = tlist[7]

In [ ]:
data.head(10)

,tweet_id,text,relevance,viewpoint,profanity,insults,char. assn.,outrage,L2_intolerance,discrimination,intolerance,incivility
0,1255911223251484672,"@sandynigeluca @JoeSchm19964270 @CNN Yes, it i...",no,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1254835941241323520,@MetallicaDad @arjay_bob @jhb430 @thehill I be...,yes,diagnostic,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1255179026123165697,@Dr_Ezekwesili LMFAO! We thank God for growth ...,no,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1255277756990066692,@ProgressiveAnd2 @204cRuiz @FlyThaiMMA @niktay...,no,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1255300688101412866,"@dahlingitsme True, I was tweeting from inside...",no,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1255293307720290305,NaN,no,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1256603811432542209,@Davien316 @russell2519 @SteveKrak When Shep S...,no,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,1253692648013074432,@PressSec @realDonaldTrump Which part of this ...,no,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8,1255290607905443840,@beamailuc @Jasonau80 @NaroomaNews Oh no the k...,no,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1254495477950750720,@DomoWarren @politicalbae_ I’m pretty sure Pue...,yes,none,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
data.to_csv("conversation_with_incivility_intolerance.csv")